In [ ]:
%matplotlib inline
from pprint import pprint
from decimal import Decimal
import pandas

import plot
import simulate
import harvesting
import market
import withdrawal
import metrics

First we want to calculate the Maximum Safe Withdrawal Rate (MSWR) for every length of retirement from 1 year up to 60 years.

In [ ]:
US = market.Returns_US_1871()

In [ ]:
def find_mswr(length, stock_pct=Decimal('.6')):
    bond_pct = 1 - stock_pct
    mswrs = []
    for start_year in range(US.start_year, 2017 - length):
        returns = []
        for annual_returns in US.iter_from(start_year, length=length):
                stocks = ((1 + annual_returns.stocks) / (1 + annual_returns.inflation) - 1)
                bonds = ((1 + annual_returns.bonds) / (1 + annual_returns.inflation) - 1)
                returns.append((stocks * stock_pct) + (bonds * (1-stock_pct)))
        pwa = metrics.pwa(1, 0, returns)
        mswrs.append(pwa)
    return min(mswrs)

In [ ]:
def find_all_mswrs():
    s = pandas.Series()
    for length in range(1, 61):
        mswr = find_mswr(length)
        s.loc[length] = mswr
    return s

In [ ]:
MSWRs = find_all_mswrs()

In [ ]:
YEARS = 40
RATE = Decimal('.0325')

Now we want to re-run the simulation. But every year, we want see if our current withdrawal rate exceeds the MSWR for our remaining retirement length. That is, when we have 14 years left, is our current withdrawal rate exceeding the MSWR for 14-year retirement?

In [ ]:
def run_one(start_year):
    yearlies = simulate.withdrawals(US.iter_from(start_year),
                         years=YEARS,
                         harvesting=harvesting.N_60_RebalanceHarvesting,
                         withdraw=lambda p, h: withdrawal.ConstantDollar(p, h, rate=RATE))
    withdraw_pct = [r.withdraw_pct_cur for r in yearlies]

    total = 0
    failed = 0
    for i in range(1, YEARS+1):
        actual = withdraw_pct[i-1]
        maximum = MSWRs.loc[YEARS - i + 1]
        if actual > maximum:
            failed += 1
        total += 1
    return (failed, total)

failed = 0
total = 0
for year in range(US.start_year, 2017-YEARS):
    f, t = run_one(year)
    print(year, f, t)
#    if f >= 1:
#        failed += 1
#    total += 1
    failed += f
    total += t
print(failed, total, failed/total)

In [ ]:
def fmt_pct(p):
    pflt = float(p) * 100
    return ('%.1f%%' % pflt)

In [ ]:
RETIRE_YEAR = 1940

In [ ]:
yearlies = simulate.withdrawals(US.iter_from(RETIRE_YEAR),
                     years=YEARS,
                     harvesting=harvesting.N_60_RebalanceHarvesting,
                     withdraw=lambda p, h: withdrawal.ConstantDollar(p, h, rate=RATE))
withdraw_pct = [('%.1f%%' % (float(r.withdraw_pct_cur)*100), '{:,.0f}'.format(r.withdraw_n), '{:,.0f}'.format(r.portfolio_n)) for r in yearlies]
df = pandas.DataFrame(withdraw_pct, columns=['Withdraw %', 'Withdraw $', 'Portfolio $'])
df.head(5)

In [ ]:
withdraw_pct = [(fmt_pct(r.withdraw_pct_cur), y) for r, y in zip(yearlies, range(YEARS, 0, -1))]
df = pandas.DataFrame(withdraw_pct, columns=['Withdraw %', 'Years To Go'])
df.head(5)

In [ ]:
withdraw_pct = [(fmt_pct(r.withdraw_pct_cur), fmt_pct(MSWRs.loc[y+1]), y) for r, y in zip(yearlies, range(YEARS, 0, -1))]
df = pandas.DataFrame(withdraw_pct, columns=['Withdraw %', 'MSWR', 'Years To Go'])
df.head(5)

In [ ]:
df.loc[df['Withdraw %'] > df['MSWR']]

In [ ]:
plot.plot([r.portfolio_r for r in yearlies], title='%s Retirement' % RETIRE_YEAR, y_label='Portfolio $')

In [ ]:
plot.plot([r.withdraw_pct_cur for r in yearlies], title='%s Retirement' % RETIRE_YEAR, y_label='Withdrawal %', add_commas=False)